In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os

print("TensorFlow Version:", tf.__version__)

TensorFlow Version: 2.20.0


In [10]:
# Define paths
base_dir = './dataset/'

# Image parameters
IMG_HEIGHT = 150
IMG_WIDTH = 150
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,      # Increased from 40
    width_shift_range=0.25,   # Increased from 0.2
    height_shift_range=0.25,  # Increased from 0.2
    shear_range=0.25,       # Increased from 0.2
    zoom_range=0.25,        # Increased from 0.2
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

# Flow training images in batches using the generator
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training' # Set as training data
)

# Flow validation images
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation' # Set as validation data
)

# Get class labels
class_labels = list(train_generator.class_indices.keys())
print("Class Labels:", class_labels)

Found 4571 images belonging to 4 classes.
Found 1141 images belonging to 4 classes.
Class Labels: ['glioma', 'meningioma', 'notumor', 'pituitary']


In [11]:
# Cell 3 (NEW): Build the Model using Transfer Learning with VGG16

from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input

# Load the VGG16 model, pre-trained on ImageNet
# include_top=False means we don't include the final classification layer
# input_shape should match our images
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# Freeze the convolutional base
# This prevents the learned weights of VGG16 from being updated during training
base_model.trainable = False

# Build our new model on top
model = Sequential([
    base_model,
    Flatten(),
    # Add our own classifier
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax') # Our 4 output classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 4, 4, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │     4,194,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,911,556 (72.14 MB)

 Trainable params: 4,196,868 (16.01 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [12]:
# Train the model
# For better accuracy, you might need more epochs (e.g., 25-50).
epochs = 37
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

Epoch 1/37
142/142 ━━━━━━━━━━━━━━━━━━━━ 113s 794ms/step - accuracy: 0.6605 - loss: 0.9078 - val_accuracy: 0.6670 - val_loss: 0.9175
Epoch 2/37
142/142 ━━━━━━━━━━━━━━━━━━━━ 26s 181ms/step - accuracy: 0.7812 - loss: 0.5463 - val_accuracy: 0.6777 - val_loss: 0.8640
Epoch 3/37
142/142 ━━━━━━━━━━━━━━━━━━━━ 135s 949ms/step - accuracy: 0.7709 - loss: 0.5972 - val_accuracy: 0.6902 - val_loss: 0.8340
Epoch 4/37
142/142 ━━━━━━━━━━━━━━━━━━━━ 26s 183ms/step - accuracy: 0.8125 - loss: 0.5019 - val_accuracy: 0.6804 - val_loss: 0.8195
Epoch 5/37
142/142 ━━━━━━━━━━━━━━━━━━━━ 126s 885ms/step - accuracy: 0.7819 - loss: 0.5553 - val_accuracy: 0.6768 - val_loss: 0.7921
Epoch 6/37
142/142 ━━━━━━━━━━━━━━━━━━━━ 25s 174ms/step - accuracy: 0.9062 - loss: 0.2954 - val_accuracy: 0.6679 - val_loss: 0.8567
Epoch 7/37
142/142 ━━━━━━━━━━━━━━━━━━━━ 123s 867ms/step - accuracy: 0.7887 - loss: 0.5269 - val_accuracy: 0.7384 - val_loss: 0.6984
Epoch 8/37
142/142 ━━━━━━━━━━━━━━━━━━━━ 25s 174ms/step - accuracy: 0.7500 - los

In [13]:
# Save the model to the model directory
model_path = '../model/brain_tumor_model.h5'
model.save(model_path)

print(f"Model saved to {model_path}")

Model saved to ../model/brain_tumor_model.h5
